# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
import os
print(os.getcwd())

/Users/zoebennett/Desktop/Analysis_Projects/Week_6(APIs)/python-api-challenge/starter_code


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    size = 'Humidity'
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[
                (city_data_df['Max Temp'] <= 24) & 
                (city_data_df['Cloudiness'] <= 40) & 
                (city_data_df['Wind Speed'] <= 3)]


# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
42,42,margate,51.3813,1.3862,16.84,76,3,1.34,GB,1666108256
...,...,...,...,...,...,...,...,...,...,...
546,546,arcata,40.8665,-124.0828,10.66,93,0,1.54,US,1666108260
558,558,batagay-alyta,67.8006,130.4114,-18.52,89,19,2.04,RU,1666108555
560,560,changli,39.7042,119.1506,9.17,33,0,2.41,CN,1666108557
564,564,redmond,47.6740,-122.1215,13.17,91,8,0.17,US,1666108556


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy(deep = True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
hotel_df = hotel_df[['Hotel Name', 'City', 'Lat', 'Lng', 'Country', 'Humidity']]

# Display sample data
hotel_df


,Hotel Name,City,Lat,Lng,Country,Humidity
0,,faya,18.3851,42.4509,SA,35
2,,new norfolk,-42.7826,147.0587,AU,58
6,,albany,42.6001,-73.9662,US,74
20,,avarua,-21.2078,-159.7750,CK,73
42,,margate,51.3813,1.3862,GB,76
...,...,...,...,...,...,...
546,,arcata,40.8665,-124.0828,US,93
558,,batagay-alyta,67.8006,130.4114,RU,89
560,,changli,39.7042,119.1506,CN,33
564,,redmond,47.6740,-122.1215,US,91


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
   'apiKey' : geoapify_key,
    'categories' : 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{Lng},{Lat},{radius}'
    params["bias"] = f'proximity:{Lng},{Lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
albany - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
margate - nearest hotel: The Reading Rooms
ancud - nearest hotel: Cabañas
lasa - nearest hotel: Hotel "Drei Kreuz"
vaini - nearest hotel: Keleti International Resort
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
provideniya - nearest hotel: No hotel found
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
cedar city - nearest hotel: Baymont Inn & Suites
lichuan - nearest hotel: No hotel found
jashpurnagar - nearest hotel: No hotel found
half moon bay - nearest hotel: Half Moon Bay Inn
corowa - nearest hotel: No hotel found
tawang - nearest hotel: Zambhala Hotel
mount gambier - nearest hotel: The Old Mount Gambier Gaol
lavrentiya - nearest hotel: Фортуна
geraldton - nearest hotel: Ocean Centre Hotel
valparaiso -

,Hotel Name,City,Lat,Lng,Country,Humidity
0,No hotel found,faya,18.3851,42.4509,SA,35
2,Woodbridge on the Derwent,new norfolk,-42.7826,147.0587,AU,58
6,No hotel found,albany,42.6001,-73.9662,US,74
20,Paradise Inn,avarua,-21.2078,-159.7750,CK,73
42,The Reading Rooms,margate,51.3813,1.3862,GB,76
...,...,...,...,...,...,...
546,Hotel Arcata,arcata,40.8665,-124.0828,US,93
558,No hotel found,batagay-alyta,67.8006,130.4114,RU,89
560,客房,changli,39.7042,119.1506,CN,33
564,Residence Inn Seattle East/Redmond,redmond,47.6740,-122.1215,US,91


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)